### Konstantina Andronikou 

## This notebook is testing the conjuction "and"

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import csv

In [3]:
from checklist.pred_wrapper import PredictorWrapper

In [4]:
# To remove all warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

srl_predictor = load_predictor('structured-prediction-srl-bert')
output = srl_predictor.predict("Someone killed Mary Smith and left.")
output

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'verbs': [{'verb': 'killed',
   'description': '[ARG0: Someone] [V: killed] [ARG1: Mary Smith] and left .',
   'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']},
  {'verb': 'left',
   'description': '[ARG0: Someone] killed Mary Smith and [V: left] .',
   'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}],
 'words': ['Someone', 'killed', 'Mary', 'Smith', 'and', 'left', '.']}

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone killed Mary Smith and left."]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'killed',
     'description': '[ARG0: Someone] [V: killed] [ARG1: Mary Smith] and left .',
     'tags': ['B-ARG0', 'B-V', 'B-ARG1', 'I-ARG1', 'O', 'O', 'O']},
    {'verb': 'left',
     'description': '[ARG0: Someone] killed Mary Smith and [V: left] .',
     'tags': ['B-ARG0', 'O', 'O', 'O', 'O', 'B-V', 'O']}],
   'words': ['Someone', 'killed', 'Mary', 'Smith', 'and', 'left', '.']}],
 array([1.]))

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg1_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name'],meta['last_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [11]:

editor = Editor()

# create examples
t = editor.template('Someone killed {first_name} {last_name} and left.', meta=True, nsamples=1000, remove_duplicates = True)

for k, v in t.items():
    print(k,v)

meta [{'first_name': 'James', 'last_name': 'Bennett'}, {'first_name': 'Louis', 'last_name': 'Brooks'}, {'first_name': 'Colin', 'last_name': 'Howard'}, {'first_name': 'Fiona', 'last_name': 'Thomas'}, {'first_name': 'Jane', 'last_name': 'Watson'}, {'first_name': 'Evelyn', 'last_name': 'Bailey'}, {'first_name': 'Katherine', 'last_name': 'Ward'}, {'first_name': 'Cynthia', 'last_name': 'Allen'}, {'first_name': 'James', 'last_name': 'Kennedy'}, {'first_name': 'Catherine', 'last_name': 'Young'}, {'first_name': 'Mary', 'last_name': 'Jordan'}, {'first_name': 'Bill', 'last_name': 'Kennedy'}, {'first_name': 'Ruth', 'last_name': 'Martin'}, {'first_name': 'Alex', 'last_name': 'Jones'}, {'first_name': 'Dave', 'last_name': 'Gray'}, {'first_name': 'Claire', 'last_name': 'Kennedy'}, {'first_name': 'Virginia', 'last_name': 'Walker'}, {'first_name': 'Catherine', 'last_name': 'Harrison'}, {'first_name': 'Hugh', 'last_name': 'Hall'}, {'first_name': 'Greg', 'last_name': 'Clark'}, {'first_name': 'Benjamin', 

In [12]:
with open('Dataset_BERT/With_Conjuction.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
test = MFT(**t, name = 'detect_arg1name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BERT/With_Conjuction_BERT.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    5 (0.5%)

Example fails:
[ARG0: Someone] [V: killed] [ARG1: Bill] Rose and left .
----
[ARG0: Someone] [V: killed] [ARG1: Evelyn] Rose and left .
----
[ARG0: Someone] [V: killed] [ARG1: Melissa] Rose and left .
----
['Someone', 'killed', 'James', 'Bennett', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Louis', 'Brooks', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Colin', 'Howard', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Fiona', 'Thomas', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Jane', 'Watson', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Evelyn', 'Bailey', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Katherine', 'Ward', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Cynthia', 'Allen', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'James', 'Kennedy', 'and', 'left', '.'] [ True]
['Someone', 'killed', 'Catherine', 'Young', 'and', 'left', '.'] [ True]
['Someone', 'killed', 

In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BERT/false_sentences_with_con_bert.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 